# Engenharia do Conhecimento 2022/2023

## TP05: A Brief introduction to Naive Bayes 

*A Machine Learning Tutorial by Andre Falcao (DI/FCUL 2020-2022)*
*revised by Catia Pesquita (DI/FCUL 2022-2023)*

### Summary

1. Introduction to Naive-Bayes using Categorical Naive Bayes
2. Naive Bayes in Scikit-learn
3. Gaussian Naive-Bayes


## 1. Introduction to Naïve Bayes

Let's start with a very simple example. We have a dataset of 14 individuals who underwent rapid tests for Covid, from different brands (Teste_A and Teste_B) These individuals were then evaluated with an effective PCR test that actually said whether or not they were carriers of the SARS-Cov2 Virus.

The results are in the covid.txt file 

In [1]:
import numpy as np
import pandas as pd
df_covid=pd.read_csv("covid.txt", sep="\t")

df_covid

,Teste_A,Teste_B,Covid
0,P,P,P
1,P,P,P
2,P,N,N
3,N,N,N
4,N,N,N
5,N,P,N
6,N,P,P
7,N,P,N
8,P,P,P
9,N,N,N


### 1.1. Basic concepts and Bayes' Theorem

Let's try to answer the following questions:
1. What is the prevalence of infected with covid?
2. What is the probability that an individual who tested positive in Test B is actually positive for Covid
3. What is the probability that an individual who has Covid will be positive on the B test? 

#### 1.1.1 prevalence of Covid
The first question we can ask is to identify the prevalence of COVID in the sample

$P(Covid_+)=\frac{N_{pos}}{N}$

The prevalence of a given class is also called *prior* because it has to do with our belief *a priori* of the proportion of individuals of each class in the population, not knowing anything else about each individual 

In [2]:
N=df_covid.shape[0] #number of instances or individuals
N_covid=df_covid[ df_covid.Covid=="P"].index.shape[0] #number of positive individuals
N_ncovid=df_covid[df_covid.Covid=="N"].index.shape[0] #number of negative individuals
P_covid=N_covid/N
P_ncovid=N_ncovid/N

print("Prevalence of positives is: %7.4f" %P_covid)
print("Prevalence of negatives is: %7.4f" %P_ncovid)


Prevalence of positives is:  0.3571
Prevalence of negatives is:  0.6429



#### 1.1.2. Identify relationship between having a test result and diagnosis

What is the probability that an individual who tested positive on the A test is in fact positive for Covid?

What we are defining is the probability of an individual being positive if the test is positive, that is, among all individuals who had a positive A test, what fraction were in fact positive:

$P(Covid_+ | A_+)=\frac{P(Covid_+ \cap A_+)}{P(A_+)}$

$P(Covid_- | A_+)=\frac{P(Covid_- \cap A_+)}{P(A_+)}$

$P(A_+)$

$P(Covid_- \cap A_+)$

What we are actually saying here is **how much will I update my conviction** that the individual is Positive **if I actually have the evidence** that test A was positive

In [3]:
#number of positive individuals with a positive test A
N_covid_Ap =df_covid[(df_covid.Covid=="P") & (df_covid.Teste_A=="P")].index.shape[0] 

#number of negative individuals with a positive test A
N_ncovid_Ap=df_covid[(df_covid.Covid=="N") & (df_covid.Teste_A=="P")].index.shape[0]

#number of individuals with a positive test A
N_Ap= df_covid[(df_covid.Teste_A=="P")].index.shape[0]

print("The probability that someone who had the A Positive test           HAS Covid is %7.4f" % (N_covid_Ap/N_Ap))
print("The probability that someone who had the A Positive test DOES NOT HAVE Covid is %7.4f" % (N_ncovid_Ap/N_Ap))


The probability that someone who had the A Positive test           HAS Covid is  0.8000
The probability that someone who had the A Positive test DOES NOT HAVE Covid is  0.2000


For Test B the procedure is the same, but even then if it seems that test B is "useless", notice that our prior conviction gets updated from 0.357 to 0.500, which is actually substantial

In [4]:
N_covid_Bp =df_covid[(df_covid.Covid=="P") & (df_covid.Teste_B=="P")].index.shape[0]
N_ncovid_Bp=df_covid[(df_covid.Covid=="N") & (df_covid.Teste_B=="P")].index.shape[0]
N_Bp= df_covid[(df_covid.Teste_B=="P")].index.shape[0]


print("The probability that someone who had the B Positive test           HAS Covid is %7.4f" % (N_covid_Bp/N_Bp))
print("The probability that someone who had the B Positive test DOES NOT HAVE Covid is %7.4f" % (N_ncovid_Bp/N_Bp))


The probability that someone who had the B Positive test           HAS Covid is  0.5000
The probability that someone who had the B Positive test DOES NOT HAVE Covid is  0.5000


#### 1.1.3. Identify relationship between a covid diagnosis and a test result


As an example: What is the probability that an individual who has Covid will test positive on the A test? What we are representing here is, within individuals with positive Covid, which fraction was positive in test A. So what we want to represent is 


$P(A_+|Covid_+ )=\frac{P(Covid_+ \cap A_+)}{P(Covid_+)}$

$P(A_-|Covid_+ )=\frac{P(Covid_+ \cap A_-)}{P(Covid_+)}$



In [5]:
#number of positive individuals with a positive test A
N_Ap_covid=df_covid[(df_covid.Covid=="P") & (df_covid.Teste_A=="P")].index.shape[0]

#number of positive individuals with a negative test A
N_An_covid=df_covid[(df_covid.Covid=="P") & (df_covid.Teste_A=="N")].index.shape[0]

L_Ap_covid=N_Ap_covid/N_covid #probability of testing positive on test A if you have covid
L_An_covid=N_An_covid/N_covid #probability of testing negative on test A if you have covid

print("The probability that someone who HAS Covid will test A POSITIVE is : %7.4f" %L_Ap_covid)
print("The probability that someone who HAS Covid will test A NEGATIVE is : %7.4f" %L_An_covid)


The probability that someone who HAS Covid will test A POSITIVE is :  0.8000
The probability that someone who HAS Covid will test A NEGATIVE is :  0.2000


Similarly for test B, we may have

In [6]:
N_Bp_covid=df_covid[(df_covid.Covid=="P") & (df_covid.Teste_B=="P")].index.shape[0]
N_Bn_covid=df_covid[(df_covid.Covid=="P") & (df_covid.Teste_B=="N")].index.shape[0]

L_Bp_covid = N_Bp_covid/N_covid
L_Bn_covid = N_Bn_covid/N_covid

print("The probability that someone who HAS Covid will test B POSITIVE is : %7.4f" % L_Bp_covid)
print("The probability that someone who HAS Covid will test B NEGATIVE is : %7.4f" % L_Bn_covid)


The probability that someone who HAS Covid will test B POSITIVE is :  0.8000
The probability that someone who HAS Covid will test B NEGATIVE is :  0.2000


#### 1.1.4. Bayes Theorem

The values above are the Likelihoods and are related to the Posterior by the union of both equations. For the case of the positive A test and positive Covid, we have the likelihood:

$P(A_+|Covid_+ )=\dfrac{P(Covid_+ \cap A_+)}{P(Covid_+)}$

and also the posterior:

$P(Covid_+ | A_+)=\dfrac{P(Covid_+ \cap A_+)}{P(A_+)}$


Therefore, it is not difficult to relate the two terms, being able to calculate the posterior according to the likelihood and the priors of the classes and categories of the variable:

$P(Covid_+ | A_+)=\dfrac{P( A_+ | Covid_+ ).P(Covid_+)}{P(A_+)}$

This last expression is one of the most common representations of Bayes' Theorem 

In [7]:
#priors of each class
P_Ap= N_Ap/N #probability of having a positive test A
P_Bp= N_Bp/N #probability of having a positive test B

#Posteriors computed according to Bayes
P_Covid_Ap = (L_Ap_covid * P_covid)/P_Ap #probability of having COVID with a positive on Test A
P_Covid_Bp = (L_Bp_covid * P_covid)/P_Bp  #probability of having COVID with a positive on Test B

print("The calculated Posterior of having Covid having a POSITIVE in test A is:  %7.4f" %P_Covid_Ap)
print("The calculated Posterior of having Covid having a POSITIVE in test B is:  %7.4f" %P_Covid_Bp)

The calculated Posterior of having Covid having a POSITIVE in test A is:   0.8000
The calculated Posterior of having Covid having a POSITIVE in test B is:   0.5000


### 1.2. Naive Bayes

The central idea in Naive Bayes is that as we can never have a population large enough to calculate the marginal probabilities of all combinations of factors, and **if we assume that the variables are independent**, we can multiply the likelihoods of the variables involved and apply the prior of the class to obtain an estimate of the *posterior* and choose among the obtained values, which one gives the highest value (Maximum a posteriori - MAP)

So for the above problem, the probability of someone having a Positive on the 2 tests, assuming that test A and test B are independent is

Probability that someone **has in fact** Covid:

$P(Covid_+ | A_+, B_+) \propto P(A_+ |Covid_+ ).P(B_+ |Covid_+ ).P(Covid_+)$

Probability that someone **does not have**  Covid:


$P(Covid_- | A_+, B_+) \propto P(A_+ |Covid_- ).P(B_+ |Covid_- ).P(Covid_-)$



In [8]:
#we will first need to calculate the likelihood associated with not having covid and having positive tests 

#number of individuals who do not have covid and test positive on test A
N_Ap_ncovid=df_covid[(df_covid.Covid=="N") & (df_covid.Teste_A=="P")].index.shape[0]  
print("individuals who do not have covid and test positive on test A: ", N_Ap_ncovid)

#number of individuals who do not have covid and test positive on test B
N_Bp_ncovid=df_covid[(df_covid.Covid=="N") & (df_covid.Teste_B=="P")].index.shape[0]
print("individuals who do not have covid and test positive on test B: ", N_Bp_ncovid)

L_Ap_ncovid=N_Ap_ncovid/N_ncovid #probability(likelihood) of having a positive on test A and not having covid
L_Bp_ncovid=N_Bp_ncovid/N_ncovid #probability(likelihood) of having a positive on test B and not having covid

P_pp_covid =  L_Ap_covid  *  L_Bp_covid   * P_covid #probability of testing positive on both tests and having covid
P_pp_ncovid = L_Ap_ncovid *  L_Bp_ncovid  * P_ncovid #probability of testing positive on both tests and not having covid

#scale factor for getting probabilities
S=P_pp_covid + P_pp_ncovid

print("The probability that someone who had both positive tests           HAS Covid is : %7.4f" % (P_pp_covid/S))
print("The probability that someone who had both positive tests DOES NOT HAVE Covid is : %7.4f" % (P_pp_ncovid/S))


individuals who do not have covid and test positive on test A:  1
individuals who do not have covid and test positive on test B:  4
The probability that someone who had both positive tests           HAS Covid is :  0.8780
The probability that someone who had both positive tests DOES NOT HAVE Covid is :  0.1220


#### Exercise 1

1.1. Calculate how likely is someone to have covid if they have both negative tests?

1.2. Calculate how likely is someone to have covid if they have test A positive and test B negative 

In [9]:
#HINT: to solve the exercise, we need to calculate the likelihood of the test being negative and not having covid 
N_An_ncovid=df_covid[(df_covid.Covid=="N") & (df_covid.Teste_A=="N")].index.shape[0]
N_Bn_ncovid=df_covid[(df_covid.Covid=="N") & (df_covid.Teste_B=="N")].index.shape[0]

L_An_ncovid=N_An_ncovid/N_ncovid
L_Bn_ncovid=N_Bn_ncovid/N_ncovid



In [10]:
#solving exercise 1.1

# just compute the posteriors. uncomment and complete:
P_nn_ncovid = L_An_ncovid * L_Bn_ncovid * P_ncovid
P_nn_covid  = L_An_covid * L_Bn_covid * P_covid

#scale factor for getting probabilities
#uncomment to run:
S = P_nn_covid + P_nn_ncovid

print("The probability that someone who had both negatives tests           HAS Covid is : %7.4f" % (P_nn_covid/S))
print("The probability that someone who had both negatives tests DOES NOT HAVE Covid is : %7.4f" % (P_nn_ncovid/S))

The probability that someone who had both negatives tests           HAS Covid is :  0.0431
The probability that someone who had both negatives tests DOES NOT HAVE Covid is :  0.9569


In [11]:
#solving exercise 1.2

P_pn_ncovid = L_Ap_ncovid * L_Bn_ncovid * P_ncovid
P_pn_covid  = L_Ap_covid * L_Bn_covid * P_covid

S = P_pn_covid + P_pn_ncovid

print("The probability that someone who had A positive and B negative tests           HAS Covid is : %7.4f" % (P_pn_covid/S))
print("The probability that someone who had A positive and B negative tests DOES NOT HAVE Covid is : %7.4f" % (P_pn_ncovid/S))

The probability that someone who had A positive and B negative tests           HAS Covid is :  0.5902
The probability that someone who had A positive and B negative tests DOES NOT HAVE Covid is :  0.4098


### 1.3. A very simple Naive Bayes implementation for categorical data

As you can see, Naive Bayes only needs 2 components:
* An estimate of the prevalences (priors)
* A separate likelihood estimate for each column of the data


The "fit" of the Naive Bayes is just calculating these statistics directly from the data. 

In [12]:
def PD_NB(X, y):
    N, M=X.shape
    
    #first compute the priors
    
    #yv: the classes (N, P)
    #yc: the number of individuals in each class
    yv, yc=np.unique(y, return_counts=True) 
    
    
    #for each output class, calculate the prior as the frequency of each class. 
    #in this case 9/14 and 5/14
    priors={yv[i]: yc[i]/sum(yc) for i in range(len(yv)) }

    
    #Now the likelihoods
    #Can be read as 
    # L[i, "B" "A"] P(Xi =B | Y = A) 
    L_hoods={}
    for j in range(M): #for each feature (i.e., each test result)
        xs=np.unique(X[:,j])
        for v in xs: #for each value of the feature (i.e., P or N)
            for yi in yv: L_hoods[(j, v,  yi)]=0.0 #prepare the likelihoods dictionary, where the key is (feature, value, output class) 
    #for each output class
    for yi in yv:
        X_c = X[y==yi] #get the individuals from each class
        #...now search each individual X column
        for j in range(M):  #for each feature (i.e., each test result)
            col = X_c[:,j] #get feature values
            #vs: the values
            #cs: the counts of each value
            vs, cs = np.unique(col, return_counts=True)
            # ...for each possible value
            for i, v in enumerate(vs):
                L_hoods[(j, v,  yi)] = cs[i]/np.sum(cs) #probability of a given value
    

    return priors, L_hoods



### Displaying the model

Let's create a simplified function for viewing priors and likelyhoods

In [13]:
#this function just prints the visualization of the priors and likelihoods
def show_NB(priors, L_hoods):
    print("Priors are:")
    for yi in priors:
        print("\tP(Y = %s) = %7.4f" % (yi, priors[yi]))
    print("Likelyhoods:")
    for i, xi, yi in L_hoods:
        print("\tP(X%d = %s | Y = %s) = %7.4f" % (i, xi, yi, L_hoods[(i,xi,yi)]))

        
X = df_covid.values[:,(0,1)]
y = df_covid.values[:,2]

priors, L_hoods=PD_NB(X,y)
show_NB(priors, L_hoods)

Priors are:
	P(Y = N) =  0.6429
	P(Y = P) =  0.3571
Likelyhoods:
	P(X0 = N | Y = N) =  0.8889
	P(X0 = N | Y = P) =  0.2000
	P(X0 = P | Y = N) =  0.1111
	P(X0 = P | Y = P) =  0.8000
	P(X1 = N | Y = N) =  0.5556
	P(X1 = N | Y = P) =  0.2000
	P(X1 = P | Y = N) =  0.4444
	P(X1 = P | Y = P) =  0.8000


### 1.4. The Naive-Bayes classifier

We can use the information calculated for each of the variables (priors and likelihood) to make estimates, by simply applying the Naive Bayes simplification that assumes the independence of the variables by multiplying the priors and likelihood associated with each column of the data. 

#### Exercise 2
1. Identify in the code below by commenting each line: 
    * How the likelihoods are used
    * how the priors are used

In [14]:
#data: an individual described by their features (here test results)


def calc_posterior(data, priors, L_hoods):
    print(data)
    
    probs=np.zeros(len(priors))

    for j, yp in enumerate(priors.keys()):
        probs[j]=priors[yp]
        for i, d in enumerate(data): probs[j]*=L_hoods[(i, d, yp)]
    #scaling to 1.0
    probs=probs/np.sum(probs)
    return {yp: probs[j] for j, yp in enumerate(priors.keys())}


Assuming that we have a set of observations for which we want to calculate the probabilities of a class, we just have to see the calculated probabilities and choose the best one. 

In [15]:
data=[["N", "N"], ["N", "P"], ["P", "N"], ["P", "P"]]
for d in data:
    res =calc_posterior(d, priors, L_hoods)
    print("Case ", d)
    cur_best=(-999,"x")
    for yi in res:
        if (res[yi], yi)>cur_best: cur_best=(res[yi], yi)
        print("\tP(Y = %s | X=%s )= %7.4f"% (yi, d, res[yi]))        
    print("\t==> Best is:", cur_best[1])

['N', 'N']
Case  ['N', 'N']
	P(Y = N | X=['N', 'N'] )=  0.9569
	P(Y = P | X=['N', 'N'] )=  0.0431
	==> Best is: N
['N', 'P']
Case  ['N', 'P']
	P(Y = N | X=['N', 'P'] )=  0.8163
	P(Y = P | X=['N', 'P'] )=  0.1837
	==> Best is: N
['P', 'N']
Case  ['P', 'N']
	P(Y = N | X=['P', 'N'] )=  0.4098
	P(Y = P | X=['P', 'N'] )=  0.5902
	==> Best is: P
['P', 'P']
Case  ['P', 'P']
	P(Y = N | X=['P', 'P'] )=  0.1220
	P(Y = P | X=['P', 'P'] )=  0.8780
	==> Best is: P


### 1.5 Test Naive-Bayes with Iris data

Since our classifier only works with categorical data, we will load the iris data and discretize it 

Mote: this code implies that scikit-learn is already installed in the system. If that is not thecase, plese see below, section 2.1

In [18]:
from sklearn.datasets import load_iris

data=load_iris()
cnames=data.feature_names
X0=data.data
class_names=data.target_names
y_iris=data.target_names[data.target]
X_iris=np.empty(X0.shape,dtype="object")
X_iris[:,:]="Medium"
for i,cname in enumerate(cnames):
    q33, q67=np.quantile(X0[:,i], (0.33, 0.67))
    X_iris[X0[:,i]<q33, i]="Small"
    X_iris[X0[:,i]>q67, i]="Large"



Take notice that we have 4 variables, each discretized in 3 intervals and 3 classes. 
1. Compute how many likelyhoods do we have to calculate?
2. And how many priors?
3. is it reasonable to expect that all likelyhoods have strict positive values? Why?

In [19]:
#there are 4*3*3=36 likelyhoods to calculate (4 variable * 3 intervals * 3 classes)
#there are 3 priors, the types of flower
#not all likelyhoods are postive, some are zero, because the dataset doesn't have cases for those likelyhoods
priors, L_hoods=PD_NB(X_iris,y_iris)
show_NB(priors, L_hoods)


Priors are:
	P(Y = setosa) =  0.3333
	P(Y = versicolor) =  0.3333
	P(Y = virginica) =  0.3333
Likelyhoods:
	P(X0 = Large | Y = setosa) =  0.0000
	P(X0 = Large | Y = versicolor) =  0.2200
	P(X0 = Large | Y = virginica) =  0.6200
	P(X0 = Medium | Y = setosa) =  0.2000
	P(X0 = Medium | Y = versicolor) =  0.6800
	P(X0 = Medium | Y = virginica) =  0.3600
	P(X0 = Small | Y = setosa) =  0.8000
	P(X0 = Small | Y = versicolor) =  0.1000
	P(X0 = Small | Y = virginica) =  0.0200
	P(X1 = Large | Y = setosa) =  0.6600
	P(X1 = Large | Y = versicolor) =  0.0400
	P(X1 = Large | Y = virginica) =  0.1600
	P(X1 = Medium | Y = setosa) =  0.3200
	P(X1 = Medium | Y = versicolor) =  0.4200
	P(X1 = Medium | Y = virginica) =  0.4600
	P(X1 = Small | Y = setosa) =  0.0200
	P(X1 = Small | Y = versicolor) =  0.5400
	P(X1 = Small | Y = virginica) =  0.3800
	P(X2 = Large | Y = setosa) =  0.0000
	P(X2 = Large | Y = versicolor) =  0.0400
	P(X2 = Large | Y = virginica) =  0.8800
	P(X2 = Medium | Y = setosa) =  0.0000
	

In [20]:
data=[["Medium", "Large", "Medium", "Large"], ["Small", "Large", "Small", "Small"]]
for d in data:
    res =calc_posterior(d, priors, L_hoods)
    print("Instance: ", d)
    cur_best=(-999,"x")
    for yi in res:
        if (res[yi], yi)>cur_best: cur_best=(res[yi], yi)
        print("\tP(Y = %s | X=%s )= %7.4f"% (yi, d, res[yi]))
    print("\t==> Best is:", cur_best[1])

['Medium', 'Large', 'Medium', 'Large']
Instance:  ['Medium', 'Large', 'Medium', 'Large']
	P(Y = setosa | X=['Medium', 'Large', 'Medium', 'Large'] )=  0.0000
	P(Y = versicolor | X=['Medium', 'Large', 'Medium', 'Large'] )=  0.1411
	P(Y = virginica | X=['Medium', 'Large', 'Medium', 'Large'] )=  0.8589
	==> Best is: virginica
['Small', 'Large', 'Small', 'Small']
Instance:  ['Small', 'Large', 'Small', 'Small']
	P(Y = setosa | X=['Small', 'Large', 'Small', 'Small'] )=  1.0000
	P(Y = versicolor | X=['Small', 'Large', 'Small', 'Small'] )=  0.0000
	P(Y = virginica | X=['Small', 'Large', 'Small', 'Small'] )=  0.0000
	==> Best is: setosa


## 2. Naive Bayes in scikit-learn


A typical machine learning procedure is executed in the following way

0. Data is typically 
    * X - the set of independent variables as a numpy array
    * y - the known values for the dependent variable as a corresponding 1D numpy array 
1. The full dataset is split into training and testing sets(we will expand on this later)
2. One initializes an algorithm with a set of parameters 
    * `myModel=model(a=1, b=2,...)`
3. The model is fit with the training set:
    * `myModel.fit(X_train,y_train)`
4. Predictions are made with the testing set
    * `predictions=myModel.predict(X_test)`
5. Classification statistics are computed


For the Covid dataset we will not need a testing set as there are only four possible values, but we will follow the general procedure


###  The Covid example


#### Step 1 - get the training and testing data

Let's process the data, by getting it out of the existing pandas dataframe. It is essential for most models that the data is numeric, so we will transform our data matrix into two columns with 0, and 1. Note that we use the pandas `get_dummies()` function, with the `drop_first= argument. True` to ensure that we have only one column and when the value is positive it is 1, otherwise it is 0 

In [21]:
X_df0 = df_covid.drop(columns=["Covid"])
X_df= pd.get_dummies(X_df0, drop_first=True)
X_df.columns=X_df0.columns

y_train = df_covid.values[:, 2]
X_train = X_df.values[:,[0,1]]
print("X_train:", X_train) 
print("y_train:", y_train)

X_train: [[1 1]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [0 1]
 [0 1]
 [0 1]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [0 1]
 [0 1]]
y_train: ['P' 'P' 'N' 'N' 'N' 'N' 'P' 'N' 'P' 'N' 'P' 'N' 'N' 'N']


The testing set will be only our four possible cases (all possible combinations of tests A and B results)

In [22]:
X_test=np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
X_test

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

#### Step 2 - set up the classifier


For the current case with categorical values **only**, we can use [CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html), which treats each variable value as categorical. 

We can now build our classifier. Note that Laplace's `alpha` is used as a parameter, very close to zero, as our simple classifier does not make this correction. So we can compare the results of both methods for the 4 types of data.

In this class we can use the classifier giving the most likely class using the `.predict(X)` method. however the `.predict_proba(X)` method returns an array with the probabilities of each class. The order of the columns is the alphabetical order of the classes, so the 1st column corresponds to class `N` and the 2nd to class `P` 

In [23]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB
mdl=CategoricalNB(alpha=0.001)


#### Step 3 - Fit the model

This is the easiest part



In [24]:
mdl.fit(X_train,y_train)

#actually the fit method returns a copy of itself so we might write
mdl = mdl.fit(X_train,y_train)


#### Step 4 - Make predictions with the testing data


In [25]:
#p2=mdl.predict(X_test)
probas=mdl.predict_proba(X_test)
for i, x in enumerate(probas):
    print("Case: ", X_test[i])
    for j, c in enumerate(["N", "P"]):
        print("\tP(Y = %s | X=%s )= %7.4f"% (c, X_test[i], x[j]))


Case:  [0 0]
	P(Y = N | X=[0 0] )=  0.9569
	P(Y = P | X=[0 0] )=  0.0431
Case:  [0 1]
	P(Y = N | X=[0 1] )=  0.8162
	P(Y = P | X=[0 1] )=  0.1838
Case:  [1 0]
	P(Y = N | X=[1 0] )=  0.4099
	P(Y = P | X=[1 0] )=  0.5901
Case:  [1 1]
	P(Y = N | X=[1 1] )=  0.1221
	P(Y = P | X=[1 1] )=  0.8779


Scikit-learn Maive Bayes has of course the `.predict(X)` method that makes the classification directly, without going through the probabilities

In [26]:
preds=mdl.predict(X_test)
for i, p in enumerate(preds):
    print("Case: ", X_test[i], "-->", p)
    

Case:  [0 0] --> N
Case:  [0 1] --> N
Case:  [1 0] --> P
Case:  [1 1] --> P



In the test set, let's identify which lines are misclassified, and examine their classification probabilities by class.

#### Exercise 5

For the base Iris data (before discretization) create a model [Naive Bayes Gaussian (Gaussian NB)](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) and test it with the same data partition. Comment on the results obtained, comparing them with the discrete model 

In [35]:
#Solução Exercício 5
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=6)

mdl=GaussianNB()
mdl.fit(X_train, y_train)

preds=mdl.predict(X_test)
print("The Accuracy score is: ", accuracy_score(y_test, preds))
pd.DataFrame(confusion_matrix(y_test, preds), columns=class_names, index=class_names)

The Accuracy score is:  0.94


NameError: name 'confusion_matrix' is not defined

## 3. Gaussian Naive Bayes (optional - not evaluated)

### 3.1. The essence of Bayesian classification in 1-Dimension

Assuming a Gaussian distribution of the independent variable, In one dimension, it is very simple to make a Bayesian classifier. Given 2 classes A and B, the probability that instance i is in class A can be computed as the product of the likelyhood and the priors

$P(i \in Class_A | x_i) \propto P(x_i |i \in Class_A).P(Class_A)$

$P(i \in Class_B | x_i) \propto P(x_i |i \in Class_B).P(Class_B)$

What makes a Bayesina classifier in this condition is the simple observation that the likelyhoods are proportional to the probability distribution functions

So for instance if we have 2 gaussian distributions for class A and B, assuming identical priors, the optimal classification separation line is where both lines intersect

So if we have a class A with mean = 2.0 and std =  0.5, and class B with mean = 3.5 and std=1.5, we can compute their distributions

In [ ]:
from scipy.stats import norm

normA=norm(loc=2.0, scale=0.5)
normB=norm(loc=3.5, scale=1.5)

And actually plot them on a given interval

In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(-1, 8, 100)
yA = normA.pdf(x)
yB = normB.pdf(x)

plt.plot(x, yA,'r-', lw=1, alpha=0.6, label='ClassA')
plt.plot(x, yB,'b-', lw=1, alpha=0.6, label='ClassB')
plt.legend()
plt.grid()
plt.show()

We can actually identify the regions that belong to each class

In [ ]:
A_regions=x[yA-yB>0]
B_regions=x[yA-yB<=0]
plt.scatter(A_regions, np.ones(A_regions.size), c="b", label="Class A")
plt.scatter(B_regions, -np.ones(B_regions.size), c="r", label="Class B")
plt.legend()
plt.grid()
plt.show()

Lets visualise a 2 dimensional classification problem

In [ ]:
import pandas as pd

df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

df_train

In [ ]:
y_train=df_train["cl"]
y_train=np.array(y_train, dtype="U")
X_train=df_train.values[:,0:2]
X_train=X_train.astype(float)

y_test=df_test["cl"]
y_test=y_test.astype("U") #need to update the type
X_test=df_test.values[:,0:2]
X_test=X_test.astype(float)


And let's plot it

In [ ]:
plt.scatter(X_train[y_train=="A",0], X_train[y_train=="A",1], label="Class A")
plt.scatter(X_train[y_train=="B",0], X_train[y_train=="B",1], label="Class B")
plt.legend()
plt.grid()
plt.show()

Let's compute the global statistics

In [ ]:
m1, m2 = np.mean(X_train, axis=0)
v1, v2 = np.var(X_train, axis=0)
s1, s2 = np.sqrt(np.array([v1, v2]))

Let's check just the difference in mean dan stddev for the first variable (column = 0)

In [ ]:
cA=X_test[y_test=="A", 0]
cB=X_test[y_test=="B", 0]

mA =np.mean(cA)
mB =np.mean(cB)
sA=np.sqrt(np.var(cA))
sB=np.sqrt(np.var(cB))
print( "For class A, the mean is %6.3f and the stdev is %6.3f" % (mA, sA))
print( "For class B, the mean is %6.3f and the stdev is %6.3f" % (mB, sB))

Following the above procedure we can similarly compute the normals and check them visually

Assuming that each class is equally likely we can plot the Gaussians of both classes.

We can see that that point around -0.38 is what separates the highest likelyhoo of a new random instance belonging to a given class

* It is class A if $x > -0.32 $
* It is class B if $x\leq -0.32 $


In [ ]:
x = np.linspace(m1-3*s1, m1+3*s1, 100)
normA=norm(loc=mA, scale=sA)
normB=norm(loc=mB, scale=sB)
yA = normA.pdf(x)
yB = normB.pdf(x)

plt.plot(x, yA,'r-', lw=1, alpha=0.6, label='ClassA')
plt.plot(x, yB,'b-', lw=1, alpha=0.6, label='ClassB')
plt.axvline(-0.38)
plt.legend()
plt.grid()
plt.show()


This would be true if the priors of both classes we know they are not as it depends on the representativity of each class which could actually be computed in a very simple way, anthough numpy has other more sofisticated ways to do so


In [ ]:
nA=np.sum(y_test=="A")
nB=np.sum(y_test=="B")
priorA, priorB = nA/(nA+nB), nB/(nA+nB), 
priorA, priorB

So we now know that actually class A is much more representative than class B, and we can update our model accordingly

In [ ]:
yA = normA.pdf(x)*priorA
yB = normB.pdf(x)*priorB

plt.plot(x, yA,'r-', lw=1, alpha=0.6, label='ClassA')
plt.plot(x, yB,'b-', lw=1, alpha=0.6, label='ClassB')
plt.legend()
plt.grid()
plt.show()

we can now see that actually class A actually is much more dominant, and appearing on two areas of X

In [ ]:
A_regions=x[yA-yB>0]
B_regions=x[yA-yB<=0]
plt.scatter(A_regions, np.ones(A_regions.size), c="b", label="Class A")
plt.scatter(B_regions, -np.ones(B_regions.size), c="r", label="Class B")
plt.legend()
plt.grid()
plt.show()

Using one variable and assuming that the distribution for both classes is Gaussian this is the actually Optimal Bayesian classifier and we can actually use these for classification

Using the testing set, we just compute the respective likelihoods of both classes and multiply by the priors and the class predicted is just when the value of one class is larger than the other


In [ ]:
yA = normA.pdf(X_test[:,0])*priorA
yB = normB.pdf(X_test[:,0])*priorB
preds=np.array(["A", "B"])[np.array(yA<=yB, dtype=int)]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
print("The global accuracy is: %7.4f" % accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))

### 2.2 Combining multiple quantitative dimensions

The above describes a Gaussian Bayesian classifier. For Naive Bayes we need to do as we have done above for categorical data. We need to compute the priors and the likelyhoods for each variable and class

$P(i \in Class_A | x_i) \propto \prod_j{P(x_{ij} |i \in Class_A)}.P(Class_A)$

$P(i \in Class_B | x_i)  \propto \prod_j{P(x_{ij} |i \in Class_B)}.P(Class_B)$

Therefore we need some functions that, given any dataset, are able to:

* compute overal statistics for parametrization of the Gaussians and priors `get_class_stats()`
* compute the Gaussian functions given the stats for each class `get_gaussians()`
* Compute the priors given the priors `get_priors()`

In [ ]:
#this function will compute the means and std deviations for all classes
def get_class_stats(X, y):
    classes=list(set(y))
    N,M=X.shape
    stats={c: None for c in classes}
    for c in classes:
        mX = X[y==c,:]
        means = np.mean(mX, axis=0)
        stds  = np.sqrt(np.var(mX, axis=0))
        count,_=mX.shape
        stats[c]=(count, means, stds)
    return stats
    
#this function will get all the gaussians: One for each variable, class combination
def get_gaussians(stats):
    gaussians={}
    for c in stats:
        N, means, stds=stats[c]
        gaussians[c]=[norm(loc=mean, scale=stds[i]) for i, mean in enumerate(means)]
    return gaussians

def get_priors(stats):
    priors = {}
    total  = 0
    for c in stats:
        N, means, stds=stats[c]
        priors[c]=N
        total+=N
    for c in priors: priors[c]/=total
    return priors


We will further require a function that given a dataset, the priors and the gaussians is able to make a prediction for each row of the dataset

In [ ]:
def gnb_predict(Xtst, priors, gaussians):
    N,M = Xtst.shape
    res         = -np.ones(N)
    final_preds = -np.zeros(N)
    final_preds=final_preds.astype(str)
    for c in gaussians:
        preds=np.ones(N)*priors[c]
        for col in range(M): preds*=gaussians[c][col].pdf(Xtst[:,col])
        final_preds[preds>res]=c
        res[preds>res]=preds[preds>res]
    return final_preds


Here is a sample use for our very simple example

In [ ]:
stats = get_class_stats(X_train,y_train)
gaussians= get_gaussians(stats)
priors = get_priors(stats)

preds = gnb_predict(X_test, priors, gaussians)
print("The global accuracy is: %7.4f" % accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))

We can also try the Iris and see how our classifier works

In [ ]:
from sklearn.model_selection import train_test_split

X_iris, y_iris = load_iris(return_X_y = True) 

X_train, X_test, y_train, y_test = train_test_split(X_iris,y_iris, test_size=0.33, random_state=22)

stats = get_class_stats(X_train,y_train)
gaussians= get_gaussians(stats)
priors = get_priors(stats)
preds = gnb_predict(X_test, priors, gaussians)
preds=preds.astype(int)  #we need to convert the data to integers
print("The global accuracy is: %7.4f" % accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))